<a href="https://colab.research.google.com/github/AsifaBeedi/converstationmanager/blob/main/projecasifa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Groq API Demonstration Notebook

This notebook demonstrates two key use cases of the Groq API:

1.  **Conversation History Management with Summarization**: Shows how to maintain and periodically summarize a conversation history to manage context length.
2.  **JSON Schema Classification & Information Extraction**: Illustrates how to use the API's tool-calling feature to extract structured data from unstructured text based on a predefined JSON schema.

**Asifa Bandulal Beedi**


## Task 1: Managing Conversation History with Summarization
This section of the code implements a `ConversationManager` class. Its primary purpose is to handle the flow of a conversation with a language model, specifically addressing the challenge of managing growing conversation history.Key features include:*

  **Adding Messages**: Appends user and assistant messages to a list maintaining the conversation turn by turn.*

  **Periodic Summarization**: After a set number of turns (`k`), the entire conversation history is summarized into a concise system message using the Groq API. This summarized system message then replaces the detailed history, effectively reducing the input token count for subsequent API calls while retaining the core context.*  
  
   **History Truncation**: Provides methods to get a truncated version of the history based on the number of recent turns or a maximum character count, useful for managing context when summarization isn't active or for specific API call requirements.*  

 **Chat Function**: Orchestrates the process of adding a user message, calling the Groq API with the current history, adding the assistant's response, and triggering summarization when the turn count reaches the specified threshold.

## Task 2: JSON Schema Classification & Information Extraction

This section demonstrates how to use the Groq API's tool-calling capability to extract structured information from unstructured text.

The process involves:

1.  **Defining a JSON Schema**: A Python dictionary `USER_DETAILS_SCHEMA` is created to specify the structure and types of the data we want to extract (e.g., name, email, phone number, location, age).
2.  **Defining a Tool**: A list of tools is created, where each tool is a dictionary describing a function the model can "call". In this case, a single tool named `extract_user_details` is defined, linked to the `USER_DETAILS_SCHEMA` as its parameters.
3.  **Using the `extract_user_details` Function**: This function takes user text as input and calls the Groq API. It instructs the model (using a system message and `tool_choice`) to use the `extract_user_details` tool to parse the input text.
4.  **Parsing Tool Output**: The API's response includes a "tool call" object containing the extracted data formatted as a JSON string according to the schema. The function then parses this JSON string into a Python dictionary.

This approach is powerful for tasks like extracting entities, filling forms, or structuring information from natural language inputs.

In [ ]:
# ==============================================================================
# Part 0: Setup
# ==============================================================================
# Install the required libraries.
!pip install groq openai

import os
import json
from openai import OpenAI
from google.colab import userdata

# --- API Key Configuration ---
# For secure practice in Colab, we use the userdata feature.
# To set this up: Click the key icon on the left sidebar, add a new secret
# named "GROQ_API_KEY", and paste your key in the value field.

try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("API Key loaded from Colab userdata.")
except userdata.SecretNotFoundError:
    print("Secret 'GROQ_API_KEY' not found. Falling back to direct input.")
    # Fallback Method: Directly pasting the key.
    # WARNING: This exposes your key in the notebook. Use with caution.
    GROQ_API_KEY = "YOUR_GROQ_API_KEY_HERE"
    if GROQ_API_KEY == "YOUR_GROQ_API_KEY_HERE":
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! PLEASE PASTE YOUR GROQ API KEY IN THE LINE ABOVE !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    else:
        print("API Key loaded from hardcoded variable.")

# Initialize the Groq client, which is compatible with the OpenAI SDK
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1",
)

print("\nSetup Complete. Groq client is ready.")

# ==============================================================================
# Task 1: Managing Conversation History with Summarization
# ==============================================================================

class ConversationManager:
    """
    Manages a conversation history, including periodic summarization and truncation,
    using only standard Python libraries.
    """
    def __init__(self, model="llama-3.1-8b-instant", k=3):
        """
        Initializes the ConversationManager.
        Args:
            model (str): The model to use for chat and summarization.
            k (int): The number of turns after which to summarize the conversation.
        """
        self.conversation_history = []
        self.model = model
        self.k = k
        self.run_count = 0
        print(f"ConversationManager initialized. Summarization will occur every {self.k} runs.")

    def add_message(self, role, content):
        """Adds a message to the conversation history."""
        self.conversation_history.append({"role": role, "content": content})

    def get_truncated_history(self, max_turns=None, max_chars=None):
        """
        Gets a truncated version of the history based on specified limits.
        Args:
            max_turns (int, optional): The number of recent user/assistant turn pairs to keep.
            max_chars (int, optional): The maximum number of characters to keep from recent messages.
        Returns:
            list: The truncated conversation history.
        """
        if max_turns:
            # A "turn" is a user message and an assistant response, so we keep max_turns * 2 messages.
            return self.conversation_history[-max_turns * 2:]
        if max_chars:
            truncated = []
            current_chars = 0
            # Iterate backwards to get the most recent messages first.
            for message in reversed(self.conversation_history):
                message_len = len(message['content'])
                if current_chars + message_len <= max_chars:
                    truncated.insert(0, message) # Add to the beginning to maintain order
                    current_chars += message_len
                else:
                    break
            return truncated
        return self.conversation_history

    def _summarize_conversation(self):
        """Internal method to call the Groq API and summarize the current history."""
        print("\n--- SUMMARIZATION TRIGGERED ---")
        history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.conversation_history])

        prompt = (
            "You are a conversation summarizer. Create a concise summary of the following dialogue, "
            "capturing the main points and user intent. This summary will serve as the context for a "
            "continuing conversation. Provide only the summary text.\n\n"
            f"Conversation:\n{history_text}\n\nSummary:"
            )

        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.2,
                )
            summary = response.choices[0].message.content
            print(f"Original History Length: {len(self.conversation_history)} messages")
            print(f"Summary Generated: '{summary}'")

            # Replace the entire history with a system message containing the summary
            self.conversation_history = [
                {"role": "system", "content": f"This is a summary of the previous conversation: {summary}"}
                ]
            print(f"New History Length: {len(self.conversation_history)} messages")
            print("--- SUMMARIZATION COMPLETE ---\n")

        except Exception as e:
            print(f"An error occurred during summarization: {e}")

    def chat(self, user_input):
        """
        Handles a complete user-assistant turn, manages history, and triggers summarization.
        """
        self.add_message("user", user_input)

        try:
            chat_completion = client.chat.completions.create(
                messages=self.conversation_history,
                model=self.model,
                )
            assistant_response = chat_completion.choices[0].message.content
        except Exception as e:
            assistant_response = f"Sorry, an error occurred: {e}"

        self.add_message("assistant", assistant_response)

        # Increment run count and check if summarization is needed.
        self.run_count += 1
        print(f"--- Turn {self.run_count} complete ---")
        if self.run_count > 0 and self.run_count % self.k == 0:
            self._summarize_conversation()

        return assistant_response

# ==============================================================================
# Task 2: JSON Schema Classification & Information Extraction
# ==============================================================================

# 1. Define the JSON schema for the information to be extracted.
USER_DETAILS_SCHEMA = {
    "type": "object",
    "properties": {
        "name": {
            "type": "string",
            "description": "The full name of the user.",
            },
        "email": {
            "type": "string",
            "description": "The email address of the user.",
            },
        "phone_number": {
            "type": "string",
            "description": "The phone number of the user, including area or country code.",
            },
        "location": {
            "type": "string",
            "description": "The city and country of the user, e.g., 'Paris, France'."
            },
        "age": {
            "type": "integer",
            "description": "The age of the user.",
            }
        },
    "required": ["name", "email"]
    }

# 2. Define the tool for the API to use.
tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_user_details",
            "description": "Extracts user information from text according to the provided JSON schema.",
            "parameters": USER_DETAILS_SCHEMA,
            },
        }
    ]

def extract_user_details(chat_text, model="openai/gpt-oss-120b"):
    """
    Uses Groq's tool-calling feature to extract structured data from text.
    A larger model is recommended for higher accuracy in function calling.
    Args:
        chat_text (str): The input text from the user.
        model (str): The model to use for extraction.
    Returns:
        dict: A dictionary containing the extracted user details.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an information extraction expert. Your task is to extract user details from the text and format it using the 'extract_user_details' tool."},
                {"role": "user", "content": chat_text}
                    ],
            tools=tools,
            tool_choice={"type": "function", "function": {"name": "extract_user_details"}},
            temperature=0.0
            )

        # 3. Parse the structured output from the tool call.
        tool_call = response.choices[0].message.tool_calls[0]
        function_args_json = tool_call.function.arguments
        extracted_data = json.loads(function_args_json)

        return extracted_data

    except Exception as e:
        return {"error": str(e)}

# ==============================================================================
# Demonstration
# ==============================================================================

print("\n\n" + "="*60)
print("STARTING TASK 1 DEMONSTRATION: CONVERSATION MANAGEMENT")
print("="*60 + "\n")

# Initialize the manager to summarize every 3rd run.
manager = ConversationManager(k=3)

# --- Run 1 ---
print("\n--- Starting Run 1 ---")
user_q1 = "Hi, I need to book a flight to Berlin for a conference next month. I'll be flying from New York."
assistant_a1 = manager.chat(user_q1)
print(f"User: {user_q1}\nAssistant: {assistant_a1}")
print(f"Current History Length: {len(manager.conversation_history)}")

# --- Run 2 ---
print("\n--- Starting Run 2 ---")
user_q2 = "I'd prefer a direct flight if possible. What airlines operate that route?"
assistant_a2 = manager.chat(user_q2)
print(f"User: {user_q2}\nAssistant: {assistant_a2}")
print(f"Current History Length: {len(manager.conversation_history)}")

# --- Run 3 (Summarization will trigger after this run) ---
print("\n--- Starting Run 3 ---")
user_q3 = "Okay, please check prices for Lufthansa for the second week of next month."
assistant_a3 = manager.chat(user_q3)
print(f"User: {user_q3}\nAssistant: {assistant_a3}")
print(f"History state after summarization: {manager.conversation_history}")

# --- Run 4 (Conversation continues with the new, summarized context) ---
print("\n--- Starting Run 4 ---")
user_q4 = "Great. Can you also recommend a hotel near the conference center?"
assistant_a4 = manager.chat(user_q4)
print(f"User: {user_q4}\nAssistant: {assistant_a4}")
print(f"Current History Length: {len(manager.conversation_history)}")

# --- Demonstrate Truncation Options ---
print("\n\n" + "="*60)
print("DEMONSTRATING TRUNCATION OPTIONS")
print("="*60 + "\n")

print(f"Full history before truncation has {len(manager.conversation_history)} messages.")

# a. Truncate by number of turns (last 1 turn)
truncated_by_turns = manager.get_truncated_history(max_turns=1)
print(f"\n1. Truncated to last 1 turn ({len(truncated_by_turns)} messages):")
print(json.dumps(truncated_by_turns, indent=2))

# b. Truncate by character length (last 200 characters)
truncated_by_chars = manager.get_truncated_history(max_chars=200)
print(f"\n2. Truncated to last ~200 characters ({len(truncated_by_chars)} messages):")
print(json.dumps(truncated_by_chars, indent=2))


print("\n\n" + "="*60)
print("STARTING TASK 2 DEMONSTRATION: JSON EXTRACTION")
print("="*60 + "\n")

sample_chats = [
    "Hello, my name is Alice Johnson and I'm 29. My email is alice.j@email.com and I live in Sydney, Australia. My phone is +61 2 9876 5432.",
    "Please register me. I'm Bob Williams, bob.w@webmail.com. I'm based in Toronto, Canada. Call me at +1-416-555-0199 if needed. I am forty-one.",
    "Just my name and email are sufficient for now: Carol White, c.white@inbox.com.",
    "My location is Singapore. Name's David Chen, 35 years old. Phone is +65 6123 4567 and my email address is david.chen@mail.sg"
    ]

for i, chat in enumerate(sample_chats):
    print(f"--- Parsing Sample Chat {i+1} ---")
    print(f"Input Text: \"{chat}\"")
    extracted_info = extract_user_details(chat)

    # 4. Validate the output by printing the structured JSON.
    print("Extracted Information (JSON):")
    print(json.dumps(extracted_info, indent=2))
    print("\n")

print("="*60)
print("DEMONSTRATION COMPLETE")
print("="*60)

API Key loaded from Colab userdata.

Setup Complete. Groq client is ready.


STARTING TASK 1 DEMONSTRATION: CONVERSATION MANAGEMENT

ConversationManager initialized. Summarization will occur every 3 runs.

--- Starting Run 1 ---
--- Turn 1 complete ---
User: Hi, I need to book a flight to Berlin for a conference next month. I'll be flying from New York.
Assistant: To help you find the best flights to Berlin from New York, I'll need to know a few details from you. 

1. What date do you plan to travel to Berlin for the conference?
2. What date do you plan to return from Berlin?
3. Do you have a preferred airline or any specific airline in mind?
4. Are you willing to take a layover or would you prefer a non-stop flight?
5. How flexible are you with your travel dates, just in case I suggest alternative dates for cheaper flights?
6. Are you looking for economy, premium economy, business, or first class?
7. How many people will be traveling?

Once I have this information, I can give you a li